In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys
import warnings

sys.path.append('../')
sys.path.append('../../')
from DataBot.data_bot import BotMLP
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

def func(epoch):

    env = FullBoard()
    agent0 = BotMLP()
    bot0 = Bot()

    wins = []

    agent0.load_model("../../DataBot/models/mlp_bot_models_v2/model_{}.hdf5".format(epoch))

    warnings.filterwarnings('ignore')

    for ep in tqdm(range(0, 10000)):

        s, _, game_over, player_turn = env.reset()
        episode_reward = [0.0, 0.0]

        time = 0
        while time < 360 or player_turn == 0:

            time += truncnorm.rvs(-2, 2, loc=3, scale=1)
            player_turn_temp = env.get_player_turn()
            env.roll_dice()[0]
            player_turn = env.get_player_turn()

            if player_turn == player_turn_temp:

                action_list = env.get_next_states(player_turn)

                if action_list:
                    if player_turn == 0:
                        s_t = env.return_state()
                        action = agent0.act(np.array(s_t[:-2]).astype('float32'), action_list)
                    else:
                        s_t = env.return_state()
                        action = bot0.act(state=s_t, p=player_turn)

                    time += truncnorm.rvs(-2, 2, loc=3, scale=1)
                    s_, reward, game_over, player_turn_temp, game_reward = env.make_step(action)

                    episode_reward[player_turn] += reward[player_turn]
        
        if game_reward[0] > game_reward[1]:
            wins.append(1)
        else:
            wins.append(0)

    return wins

2022-10-26 19:55:21.282573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:55:21.295384: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:55:21.296300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import pickle

wins = []
epoch = []
win_p = []
for i in range(44, 1000, 5):
    temp = func(i)
    print(i, np.sum(temp))
    epoch.append(i)
    win_p.append(np.sum(temp))
    wins.append(temp)

    with open('.results.pkl', 'wb') as f:
        pickle.dump([epoch, win_p], f)

2022-10-26 19:55:21.477183: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 19:55:21.480337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:55:21.481215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:55:21.481956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [1]:
import pickle

with open('.results.pkl', 'rb') as f:
    epoch, win_p = pickle.load(f)

print(epoch, win_p)

[44, 49, 54, 59, 64, 69, 74, 79, 84, 89, 94, 99] [5350, 5296, 5141, 5364, 5082, 5450, 5475, 5319, 5614, 5365, 5457, 5456]
